In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import roc_curve,auc,log_loss, roc_auc_score
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA
import lightgbm as lgb

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [4]:
train.describe()

id      target           0           1           2           3  \
count  250.000000  250.000000  250.000000  250.000000  250.000000  250.000000   
mean   124.500000    0.640000    0.023292   -0.026872    0.167404    0.001904   
std     72.312977    0.480963    0.998354    1.009314    1.021709    1.011751   
min      0.000000    0.000000   -2.319000   -2.931000   -2.477000   -2.359000   
25%     62.250000    0.000000   -0.644750   -0.739750   -0.425250   -0.686500   
50%    124.500000    1.000000   -0.015500    0.057000    0.184000   -0.016500   
75%    186.750000    1.000000    0.677000    0.620750    0.805000    0.720000   
max    249.000000    1.000000    2.567000    2.419000    3.392000    2.771000   

                4           5           6           7  ...         290  \
count  250.000000  250.000000  250.000000  250.000000  ...  250.000000   
mean     0.001588   -0.007304    0.032052    0.078412  ...    0.044652   
std      1.035411    0.955700    1.006657    0.939731  ...    1.011416   
min     -2.566000   -2.845000   -2.976000   -3.444000  ...   -2.804000   
25%     -0.659000   -0.643750   -0.675000   -0.550750  ...   -0.617000   
50%     -0.023000    0.037500    0.060500    0.183500  ...    0.067500   
75%      0.735000    0.660500    0.783250    0.766250  ...    0.797250   
max      2.901000    2.793000    2.546000    2.846000  ...    2.865000   

              291         292         293         294         295         296  \
count  250.000000  250.000000  250.000000  250.000000  250.000000  250.000000   
mean     0.126344    0.018436   -0.012092   -0.065720   -0.106112    0.046472   
std      0.972567    0.954229    0.960630    1.057414    1.038389    0.967661   
min     -2.443000   -2.757000   -2.466000   -3.287000   -3.072000   -2.634000   
25%     -0.510500   -0.535750   -0.657000   -0.818500   -0.821000   -0.605500   
50%      0.091000    0.057500   -0.021000   -0.009000   -0.079500    0.009500   
75%      0.804250    0.631500    0.650250    0.739500    0.493000    0.683000   
max      2.801000    2.736000    2.596000    2.226000    3.131000    3.236000   

              297         298         299  
count  250.000000  250.000000  250.000000  
mean     0.006452    0.009372   -0.128952  
std      0.998984    1.008099    0.971219  
min     -2.776000   -3.211000   -3.500000  
25%     -0.751250   -0.550000   -0.754250  
50%      0.005500   -0.009000   -0.132500  
75%      0.794250    0.654250    0.503250  
max      2.626000    3.530000    2.771000  

[8 rows x 302 columns]

In [5]:
test.describe().loc[:,[i for i in l]]

NameError: name 'l' is not defined

In [ ]:
l=list(train.corr()['target'].sort_values(ascending=True)[:5].index)

In [ ]:
l

In [ ]:
train.describe().loc[:,[i for i in l]]

In [ ]:
l2=list(train.corr()['target'].sort_values(ascending=False)[1:6].index)
l2

In [ ]:
test.describe().loc[:,[i for i in l2]]

In [60]:
train.describe().loc[:,[i for i in l2]]

33          65          24         183         199
count  250.000000  250.000000  250.000000  250.000000  250.000000
mean    -0.134460    0.003796   -0.128252    0.005128   -0.079264
std      1.015563    0.983879    0.997026    0.969452    1.039407
min     -2.913000   -2.799000   -3.107000   -3.118000   -3.521000
25%     -0.829000   -0.636000   -0.779500   -0.710500   -0.853750
50%     -0.216500    0.022000   -0.165500   -0.013500   -0.017000
75%      0.557750    0.687000    0.579250    0.588750    0.674750
max      2.649000    3.163000    2.927000    2.943000    2.185000

In [68]:
pd.Series.corr(train.loc[:,'199'],test.loc[:,'199'])

0.008344638302215652

In [40]:
l2

target    1.000000
33        0.373608
65        0.293846
24        0.173096
183       0.164146
199       0.159442
Name: target, dtype: float64

In [72]:
train['target'].value_counts()

1.0    160
0.0     90
Name: target, dtype: int64

In [9]:
X=train.drop(['target','id'],axis=1,inplace=False)

In [10]:
X_test=test.drop('id',axis=1,inplace=False)

In [11]:
y=train['target']


In [12]:
X_test.shape

(19750, 300)

#### Lightgbm parameter tunung

In [73]:
param = {
   "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    'n_estimators':100,
    "num_leaves" : 13,
    "learning_rate" : 0.01,
    "bagging_freq": 5,
    "bagging_fraction" : 1,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "bagging_seed" : 10,
    "verbosity" : 1,
}  



In [78]:
params = {'boosting': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': -1, 
          'n_estimators':500,
          'num_leaves': 10,
          'learning_rate': 0.1,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 1,
          'min_child_samples': 5,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'auc'}

gridParams = {
    'learning_rate': [0.01,0.05,0.1],
    'n_estimators': [500],
    'num_leaves': [5,10,20,30,40,50,60],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66,0.5,0.8],
    'subsample' : [0.7],
    'reg_alpha' : [0.1,0.5,1,1.5,2],
    'reg_lambda' : [0.1,0.5,1,1.5,2],
    'scale_pos_weight': [0.5,1],
    'min_child_weight':[5,10,20]
    }

mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          n_estimators=params['n_estimators'],
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=10,
                    n_jobs=-1)
# Run the grid
grid.fit(X, y)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)



{'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.1, 'min_child_weight': 10, 'n_estimators': 500, 'num_leaves': 5, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'scale_pos_weight': 1, 'subsample': 0.7}
0.768


In [81]:
#param={"metric" : "auc",'boosting_type': 'gbdt', 'colsample_bytree': 0.5, 'learning_rate': 0.03, 'n_estimators': 500, 'num_leaves': 5, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 0.5, 'reg_lambda': 1.5, 'subsample': 0.7}
param={"metric" : "auc",'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.1, 'min_child_weight': 10, 'n_estimators': 500, 'num_leaves': 5, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 0.5, 'reg_lambda': 0.1, 'scale_pos_weight': 1, 'subsample': 0.7}

#### BayesianOptimization

In [20]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_test, label=y_test)
        

In [22]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = dtrain
    valid_data=dvalid
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (2, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (2, 10),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50),
                                            'learning_rate':(0.01,0.1)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO.res

#opt_params = bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=10, n_folds=3, random_seed=6, n_estimators=100, learning_rate=0.05)

In [23]:
bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=10, n_folds=3, random_seed=6, n_estimators=100) 

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------


TypeError: lgb_eval() got an unexpected keyword argument 'learning_rate'

In [47]:
param= {"objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    'bagging_fraction': 0.8075296044438405,
   'feature_fraction': 0.8791677018556472,
   'lambda_l1': 4.625088158766973,
   'lambda_l2': 2.8815506301653055,
   'max_depth': 9,
   'min_child_weight': 10.733768761460528,
   'min_split_gain': 0.0948779856087671,
   'num_leaves': 24}

In [13]:
param= {'num_leaves': 5,
 'max_bin': 63,
 'min_data_in_leaf': 6,
 'learning_rate': 0.20248587212811242,
 'min_sum_hessian_in_leaf': 0.0022989382746764404,
 'bagging_fraction': 1.0,
 'bagging_freq': 5,
 'feature_fraction': 0.16640554620032916,
 'lambda_l1': 4.974808317431579,
 'lambda_l2': 4.782641563951209,
 'min_gain_to_split': 0.9880688059367111,
 'max_depth': 3,
 'save_binary': True,
 'seed': 1337,
 'feature_fraction_seed': 1337,
 'bagging_seed': 1337,
 'drop_seed': 1337,
 'data_random_seed': 1337,
 'objective': 'binary',
 'boosting_type': 'gbdt',
 'verbose': 1,
 'metric': 'auc',
 'is_unbalance': True,
 'boost_from_average': False}

In [7]:
###train the model

In [15]:
from sklearn.model_selection import RepeatedStratifiedKFold
result=np.zeros(X_test.shape[0])

rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10,random_state=42)
for counter,(train_index, valid_index) in enumerate(rskf.split(X, y),1):
    print (counter)
    
    #Train data
    t=train.iloc[train_index]
    #t=augment(t)
    trn_data = lgb.Dataset(t.drop(["target",'id'],axis=1,inplace=False), label=t.target)
    
    #Validation data
    v=train.iloc[valid_index]
    val_data = lgb.Dataset(v.drop(["target",'id'],axis=1,inplace=False), label=v.target)
    
    #Training
    model = lgb.train(param, trn_data, 1000000, valid_sets = [val_data], verbose_eval=100, early_stopping_rounds = 3000)
    result +=model.predict(X_test)

1
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.756944
[200]	valid_0's auc: 0.756944
[300]	valid_0's auc: 0.756944
[400]	valid_0's auc: 0.756944
[500]	valid_0's auc: 0.756944
[600]	valid_0's auc: 0.756944
[700]	valid_0's auc: 0.756944
[800]	valid_0's auc: 0.756944
[900]	valid_0's auc: 0.756944
[1000]	valid_0's auc: 0.756944
[1100]	valid_0's auc: 0.756944
[1200]	valid_0's auc: 0.756944
[1300]	valid_0's auc: 0.756944
[1400]	valid_0's auc: 0.756944
[1500]	valid_0's auc: 0.756944
[1600]	valid_0's auc: 0.756944
[1700]	valid_0's auc: 0.756944
[1800]	valid_0's auc: 0.756944
[1900]	valid_0's auc: 0.756944
[2000]	valid_0's auc: 0.756944
[2100]	valid_0's auc: 0.756944
[2200]	valid_0's auc: 0.756944
[2300]	valid_0's auc: 0.756944
[2400]	valid_0's auc: 0.756944
[2500]	valid_0's auc: 0.756944
[2600]	valid_0's auc: 0.756944
[2700]	valid_0's auc: 0.756944
[2800]	valid_0's auc: 0.756944
[2900]	valid_0's auc: 0.756944
[3000]	valid_0's auc: 0.756944
Early stoppin

[3000]	valid_0's auc: 0.715278
Early stopping, best iteration is:
[25]	valid_0's auc: 0.736111
9
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.569444
[200]	valid_0's auc: 0.569444
[300]	valid_0's auc: 0.569444
[400]	valid_0's auc: 0.569444
[500]	valid_0's auc: 0.569444
[600]	valid_0's auc: 0.569444
[700]	valid_0's auc: 0.569444
[800]	valid_0's auc: 0.569444
[900]	valid_0's auc: 0.569444
[1000]	valid_0's auc: 0.569444
[1100]	valid_0's auc: 0.569444
[1200]	valid_0's auc: 0.569444
[1300]	valid_0's auc: 0.569444
[1400]	valid_0's auc: 0.569444
[1500]	valid_0's auc: 0.569444
[1600]	valid_0's auc: 0.569444
[1700]	valid_0's auc: 0.569444
[1800]	valid_0's auc: 0.569444
[1900]	valid_0's auc: 0.569444
[2000]	valid_0's auc: 0.569444
[2100]	valid_0's auc: 0.569444
[2200]	valid_0's auc: 0.569444
[2300]	valid_0's auc: 0.569444
[2400]	valid_0's auc: 0.569444
[2500]	valid_0's auc: 0.569444
[2600]	valid_0's auc: 0.569444
[2700]	valid_0's auc: 0.569444
[2800]	vali

17
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.618056
[200]	valid_0's auc: 0.618056
[300]	valid_0's auc: 0.618056
[400]	valid_0's auc: 0.618056
[500]	valid_0's auc: 0.618056
[600]	valid_0's auc: 0.618056
[700]	valid_0's auc: 0.618056
[800]	valid_0's auc: 0.618056
[900]	valid_0's auc: 0.618056
[1000]	valid_0's auc: 0.618056
[1100]	valid_0's auc: 0.618056
[1200]	valid_0's auc: 0.618056
[1300]	valid_0's auc: 0.618056
[1400]	valid_0's auc: 0.618056
[1500]	valid_0's auc: 0.618056
[1600]	valid_0's auc: 0.618056
[1700]	valid_0's auc: 0.618056
[1800]	valid_0's auc: 0.618056
[1900]	valid_0's auc: 0.618056
[2000]	valid_0's auc: 0.618056
[2100]	valid_0's auc: 0.618056
[2200]	valid_0's auc: 0.618056
[2300]	valid_0's auc: 0.618056
[2400]	valid_0's auc: 0.618056
[2500]	valid_0's auc: 0.618056
[2600]	valid_0's auc: 0.618056
[2700]	valid_0's auc: 0.618056
[2800]	valid_0's auc: 0.618056
[2900]	valid_0's auc: 0.618056
[3000]	valid_0's auc: 0.618056
Early stoppi

25
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.777778
[200]	valid_0's auc: 0.777778
[300]	valid_0's auc: 0.777778
[400]	valid_0's auc: 0.777778
[500]	valid_0's auc: 0.777778
[600]	valid_0's auc: 0.777778
[700]	valid_0's auc: 0.777778
[800]	valid_0's auc: 0.777778
[900]	valid_0's auc: 0.777778
[1000]	valid_0's auc: 0.777778
[1100]	valid_0's auc: 0.777778
[1200]	valid_0's auc: 0.777778
[1300]	valid_0's auc: 0.777778
[1400]	valid_0's auc: 0.777778
[1500]	valid_0's auc: 0.777778
[1600]	valid_0's auc: 0.777778
[1700]	valid_0's auc: 0.777778
[1800]	valid_0's auc: 0.777778
[1900]	valid_0's auc: 0.777778
[2000]	valid_0's auc: 0.777778
[2100]	valid_0's auc: 0.777778
[2200]	valid_0's auc: 0.777778
[2300]	valid_0's auc: 0.777778
[2400]	valid_0's auc: 0.777778
[2500]	valid_0's auc: 0.777778
[2600]	valid_0's auc: 0.777778
[2700]	valid_0's auc: 0.777778
[2800]	valid_0's auc: 0.777778
[2900]	valid_0's auc: 0.777778
[3000]	valid_0's auc: 0.777778
Early stoppi

33
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.729167
[200]	valid_0's auc: 0.729167
[300]	valid_0's auc: 0.729167
[400]	valid_0's auc: 0.729167
[500]	valid_0's auc: 0.729167
[600]	valid_0's auc: 0.729167
[700]	valid_0's auc: 0.729167
[800]	valid_0's auc: 0.729167
[900]	valid_0's auc: 0.729167
[1000]	valid_0's auc: 0.729167
[1100]	valid_0's auc: 0.729167
[1200]	valid_0's auc: 0.729167
[1300]	valid_0's auc: 0.729167
[1400]	valid_0's auc: 0.729167
[1500]	valid_0's auc: 0.729167
[1600]	valid_0's auc: 0.729167
[1700]	valid_0's auc: 0.729167
[1800]	valid_0's auc: 0.729167
[1900]	valid_0's auc: 0.729167
[2000]	valid_0's auc: 0.729167
[2100]	valid_0's auc: 0.729167
[2200]	valid_0's auc: 0.729167
[2300]	valid_0's auc: 0.729167
[2400]	valid_0's auc: 0.729167
[2500]	valid_0's auc: 0.729167
[2600]	valid_0's auc: 0.729167
[2700]	valid_0's auc: 0.729167
[2800]	valid_0's auc: 0.729167
[2900]	valid_0's auc: 0.729167
[3000]	valid_0's auc: 0.729167
Early stoppi

...

[2600]	valid_0's auc: 0.680556
[2700]	valid_0's auc: 0.680556
[2800]	valid_0's auc: 0.680556
[2900]	valid_0's auc: 0.680556
[3000]	valid_0's auc: 0.680556
Early stopping, best iteration is:
[8]	valid_0's auc: 0.791667
65
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.625
[200]	valid_0's auc: 0.625
[300]	valid_0's auc: 0.625
[400]	valid_0's auc: 0.625
[500]	valid_0's auc: 0.625
[600]	valid_0's auc: 0.625
[700]	valid_0's auc: 0.625
[800]	valid_0's auc: 0.625
[900]	valid_0's auc: 0.625
[1000]	valid_0's auc: 0.625
[1100]	valid_0's auc: 0.625
[1200]	valid_0's auc: 0.625
[1300]	valid_0's auc: 0.625
[1400]	valid_0's auc: 0.625
[1500]	valid_0's auc: 0.625
[1600]	valid_0's auc: 0.625
[1700]	valid_0's auc: 0.625
[1800]	valid_0's auc: 0.625
[1900]	valid_0's auc: 0.625
[2000]	valid_0's auc: 0.625
[2100]	valid_0's auc: 0.625
[2200]	valid_0's auc: 0.625
[2300]	valid_0's auc: 0.625
[2400]	valid_0's auc: 0.625
[2500]	valid_0's auc: 0.625
[2600]	valid_0's auc: 0.

[2300]	valid_0's auc: 0.680556
[2400]	valid_0's auc: 0.680556
[2500]	valid_0's auc: 0.680556
[2600]	valid_0's auc: 0.680556
[2700]	valid_0's auc: 0.680556
[2800]	valid_0's auc: 0.680556
[2900]	valid_0's auc: 0.680556
[3000]	valid_0's auc: 0.680556
Early stopping, best iteration is:
[26]	valid_0's auc: 0.729167
73
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.8125
[200]	valid_0's auc: 0.8125
[300]	valid_0's auc: 0.8125
[400]	valid_0's auc: 0.8125
[500]	valid_0's auc: 0.8125
[600]	valid_0's auc: 0.8125
[700]	valid_0's auc: 0.8125
[800]	valid_0's auc: 0.8125
[900]	valid_0's auc: 0.8125
[1000]	valid_0's auc: 0.8125
[1100]	valid_0's auc: 0.8125
[1200]	valid_0's auc: 0.8125
[1300]	valid_0's auc: 0.8125
[1400]	valid_0's auc: 0.8125
[1500]	valid_0's auc: 0.8125
[1600]	valid_0's auc: 0.8125
[1700]	valid_0's auc: 0.8125
[1800]	valid_0's auc: 0.8125
[1900]	valid_0's auc: 0.8125
[2000]	valid_0's auc: 0.8125
[2100]	valid_0's auc: 0.8125
[2200]	valid_0's auc:

[2400]	valid_0's auc: 0.75
[2500]	valid_0's auc: 0.75
[2600]	valid_0's auc: 0.75
[2700]	valid_0's auc: 0.75
[2800]	valid_0's auc: 0.75
[2900]	valid_0's auc: 0.75
[3000]	valid_0's auc: 0.75
Early stopping, best iteration is:
[31]	valid_0's auc: 0.756944
81
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.666667
[200]	valid_0's auc: 0.666667
[300]	valid_0's auc: 0.666667
[400]	valid_0's auc: 0.666667
[500]	valid_0's auc: 0.666667
[600]	valid_0's auc: 0.666667
[700]	valid_0's auc: 0.666667
[800]	valid_0's auc: 0.666667
[900]	valid_0's auc: 0.666667
[1000]	valid_0's auc: 0.666667
[1100]	valid_0's auc: 0.666667
[1200]	valid_0's auc: 0.666667
[1300]	valid_0's auc: 0.666667
[1400]	valid_0's auc: 0.666667
[1500]	valid_0's auc: 0.666667
[1600]	valid_0's auc: 0.666667
[1700]	valid_0's auc: 0.666667
[1800]	valid_0's auc: 0.666667
[1900]	valid_0's auc: 0.666667
[2000]	valid_0's auc: 0.666667
[2100]	valid_0's auc: 0.666667
[2200]	valid_0's auc: 0.666667
[2300]	

[2500]	valid_0's auc: 0.625
[2600]	valid_0's auc: 0.625
[2700]	valid_0's auc: 0.625
[2800]	valid_0's auc: 0.625
[2900]	valid_0's auc: 0.625
[3000]	valid_0's auc: 0.625
Early stopping, best iteration is:
[19]	valid_0's auc: 0.680556
89
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.756944
[200]	valid_0's auc: 0.756944
[300]	valid_0's auc: 0.756944
[400]	valid_0's auc: 0.756944
[500]	valid_0's auc: 0.756944
[600]	valid_0's auc: 0.756944
[700]	valid_0's auc: 0.756944
[800]	valid_0's auc: 0.756944
[900]	valid_0's auc: 0.756944
[1000]	valid_0's auc: 0.756944
[1100]	valid_0's auc: 0.756944
[1200]	valid_0's auc: 0.756944
[1300]	valid_0's auc: 0.756944
[1400]	valid_0's auc: 0.756944
[1500]	valid_0's auc: 0.756944
[1600]	valid_0's auc: 0.756944
[1700]	valid_0's auc: 0.756944
[1800]	valid_0's auc: 0.756944
[1900]	valid_0's auc: 0.756944
[2000]	valid_0's auc: 0.756944
[2100]	valid_0's auc: 0.756944
[2200]	valid_0's auc: 0.756944
[2300]	valid_0's auc: 0.7569

[1900]	valid_0's auc: 0.659722
[2000]	valid_0's auc: 0.659722
[2100]	valid_0's auc: 0.659722
[2200]	valid_0's auc: 0.659722
[2300]	valid_0's auc: 0.659722
[2400]	valid_0's auc: 0.659722
[2500]	valid_0's auc: 0.659722
[2600]	valid_0's auc: 0.659722
[2700]	valid_0's auc: 0.659722
[2800]	valid_0's auc: 0.659722
[2900]	valid_0's auc: 0.659722
[3000]	valid_0's auc: 0.659722
Early stopping, best iteration is:
[45]	valid_0's auc: 0.666667
97
Training until validation scores don't improve for 3000 rounds.
[100]	valid_0's auc: 0.826389
[200]	valid_0's auc: 0.826389
[300]	valid_0's auc: 0.826389
[400]	valid_0's auc: 0.826389
[500]	valid_0's auc: 0.826389
[600]	valid_0's auc: 0.826389
[700]	valid_0's auc: 0.826389
[800]	valid_0's auc: 0.826389
[900]	valid_0's auc: 0.826389
[1000]	valid_0's auc: 0.826389
[1100]	valid_0's auc: 0.826389
[1200]	valid_0's auc: 0.826389
[1300]	valid_0's auc: 0.826389
[1400]	valid_0's auc: 0.826389
[1500]	valid_0's auc: 0.826389
[1600]	valid_0's auc: 0.826389
[1700]	val

In [16]:
len(result)

19750

In [17]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = result/counter
#filename="{:%Y-%m-%d_%H_%M}_sub.csv".format(datetime.now())
submission.to_csv('submission_with_augm.csv', index=False)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Since there was no preprocessing, we didn't need a pipeline here. Used anyway as best practice
modeling_pipeline = make_pipeline(RandomForestClassifier(n_estimators =10000,max_depth=4,max_features ='sqrt',n_jobs=-1))
cv_scores = RepeatedStratifiedKFold(modeling_pipeline, X_tr, y_tr, scoring='roc_auc',cv=20)
for counter,(train_index, valid_index) in enumerate(cv_scores.split(train, train.target),1):
    
    
print("Cross-val auc : %f" %cv_scores.mean())
print("Cross-val std : %f" %cv_scores.std())
print("Cross-val :",cv_scores)